In [237]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import plotly.graph_objects as go
import networkx as nx

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import image
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from keras.layers import Embedding
from keras.models import Sequential
import tensorflow as tf
from mtcnn import MTCNN


from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model, load_model
import cv2
import os

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader, Loader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess
from spektral.utils import normalized_laplacian

import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [238]:
df = pd.read_csv('MELD.Raw/dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
df['gender'] = df.Speaker.apply(lambda x: 'male' if x == 'Ross' or x == 'Joey' or x == 'Chandler' else 'female')
df = df.sample(frac=0.01,ignore_index=True)
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,gender
0,271,"So in your whole life, you've only been with o...",Chandler,surprise,negative,22,12,1,4,"00:18:21,683","00:18:24,519",male
1,921,That's the magic story you use when you wanna ...,Chandler,anger,negative,85,10,8,4,"00:18:52,214","00:18:55,675",male
2,536,"So just stop all the yelling, just stop it!",Phoebe,anger,negative,49,11,1,23,"00:09:45,626","00:09:48,086",female
3,19,"Hey Estelle, listen",Joey,neutral,neutral,2,0,6,4,"00:08:53,241","00:08:57,224",male
4,82,And the nudity is really important to the story.,Joey,neutral,neutral,8,4,7,19,"00:03:27,332","00:03:29,750",male
5,1110,Hey Chandler! Y'know that girl you went to col...,Joey,neutral,neutral,106,0,6,18,"00:01:59,994","00:02:03,997",male
6,1118,You really think so?,Joey,surprise,positive,107,2,6,20,"0:02:44,789","0:02:45,501",male
7,736,Okay.,Tag,neutral,neutral,69,10,7,8,"00:20:07,706","00:20:09,081",female
8,724,"Oh, hi! How are you doing?",Rachel,joy,positive,69,0,7,8,"00:19:38,969","00:19:41,000",female
9,648,"Oh, I know. Y'know, just-just talking about i...",Ross,neutral,neutral,62,4,6,16,"00:01:23,124","00:01:30,964",male


In [239]:
def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]


texts = df.prepro.values
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post')

In [240]:
glove_path = 'glove.6B.50d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, encoding='utf8',no_header=True)



In [241]:
embedding_dim = 50  
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

embedding_layer = Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=50,
    trainable=False
)


model = Sequential()
model.add(embedding_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics='accuracy')


In [242]:
embedding_vectors = model.predict(padded_sequences)
df['text'] = list(embedding_vectors)

1/1 [==============================] - 0s 45ms/step


In [243]:
# Load MTCNN model for face detection
detector = MTCNN()

# Load FaceNet model for face embedding extraction
facenet_model = tf.keras.applications.ResNet50(weights='imagenet')

def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0 
    return img

def extract_face_embeddings(frame):
    faces = detector.detect_faces(frame)
    face_embeddings = []
    for face in faces:
        x, y, w, h = face['box']
        x1, y1 = max(x, 0), max(y, 0)
        x2, y2 = min(x + w, frame.shape[1]), min(y + h, frame.shape[0])
        cropped_face = frame[y1:y2, x1:x2]
        
        # Preprocess the face for FaceNet model
        preprocessed_face = preprocess_image(cropped_face)
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)
        
        # Get face embeddings using FaceNet model
        face_embedding = facenet_model.predict(preprocessed_face)
        face_embeddings.append(face_embedding)
    return face_embeddings

# Read video
folder_path = 'MELD.Raw/dev_splits_complete/'
one_face_videos = {}

for idx,row in df.iterrows():
    file_name = 'dia' + str(row['Dialogue_ID']) + '_utt' + str(row['Utterance_ID']) + '.mp4'
    video_path = folder_path + file_name
    if os.path.isfile(video_path):
        video_capture = cv2.VideoCapture(video_path)  # Provide the path to your input video

        single_video_embeddings = []  # List to store embeddings for all frames

        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            #check for more than 1 faces
            if len(detector.detect_faces(frame)) == 1:
            
                # Extract face embeddings from each frame
                extracted_embeddings = extract_face_embeddings(frame)
                
                single_video_embeddings.append(extracted_embeddings)  # Append embeddings for this frame
                
                # Display the frame with bounding boxes around detected faces
                for face in detector.detect_faces(frame):
                    x, y, w, h = face['box']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                cv2.imshow('Video', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                video_capture.release()
                cv2.destroyAllWindows()

            else:
                break
        if single_video_embeddings != []:            
            one_face_videos[file_name] = single_video_embeddings
        else:
            one_face_videos[file_name] = 'too many faces'

                




1/1 [==============================] - 0s 19ms/step


In [244]:
df['faces'] = one_face_videos.values()

df = df[df.faces != 'too many faces']

df['faces'] = df.faces.apply(lambda x: x[0][0][0])


C:\Users\gsevr\AppData\Local\Temp\ipykernel_14060\1239130037.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [245]:
G = nx.Graph()


In [246]:

emotion_col = {
            'sadness':'teal',
            'surprise':'silver',
            'neutral':'lightgreen',
            'joy':'purple',
            'anger':'black',
            'disgust': 'brown',
            'fear':'yellow'
}

gender_col = {
    'male':'blue',
    'female':'pink'
}

In [247]:
for emotion in df.Emotion.unique():
    G.add_node(emotion, color=emotion_col[emotion],hover=emotion,emotion=emotion, type='emotion')


In [248]:
for gender in df.gender.unique():
    G.add_node(gender,color=gender_col[gender], hover=gender, gender=gender, type='gender')

In [249]:
for idx, row in df.iterrows():
    G.add_node(row['Utterance'], 
    text_embedding=row['text'],
    visual_embedding=row['faces'],
    color=emotion_col[row['Emotion']],
    hover=row['Utterance'],
    emotion=row['Emotion'],
    gender=row['gender'], 
    type='utterance',
    padding='zero')

In [250]:
for idx, row in df.iterrows():
    G.add_edge(row['Utterance'], row['Emotion'],color=emotion_col[row['Emotion']],weight=1)
    G.add_edge(row['Utterance'], row['gender'], color=gender_col[row['gender']],weight=1)


In [251]:
pos = nx.spring_layout(G)
# pos = nx.fruchterman_reingold_layout(G)


In [252]:
import textwrap

fig = go.Figure()

node_positions = {}

for node, position in pos.items():
    x, y = position  

    color = G.nodes[node]['color']  
    hover = G.nodes[node]['hover']
    hover = '<br>'.join(textwrap.wrap(hover,width=50))
    try:
        emotion = G.nodes[node]['emotion']
    except KeyError:
        try:
            gender = G.nodes[node]['gender']
        except KeyError:
            continue
    node_type = G.nodes[node]['type']

    if node_type == 'utterance':
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=emotion, hovertemplate=hover))
    else:
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=node_type, hovertemplate=hover))


for edge in G.edges():
    color = G.edges[edge]['color']
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(width=1,color=color), name='Edge'))

fig.update_layout(showlegend=False, title='Emotions Graph')

fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

fig.show()

In [253]:
# Adj matrix

adjacency_info = dict(G.adj)

nodes = list(G.nodes())

adj_matrix = np.zeros((len(nodes), len(nodes)))

for i, node in enumerate(nodes):
    for j, other_node in enumerate(nodes):
        if other_node in adjacency_info[node]:
            adj_matrix[i, j] = 1 

print(adj_matrix)


[[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [254]:
# try spektral

adjacency_matrix = nx.adjacency_matrix(G)  # Adjacency matrix
features_text = np.array([G.nodes[node]['text_embedding'] for node in G.nodes() if G.nodes[node]['type'] == 'utterance'])  # 'text' features
features_video = np.array([G.nodes[node]['visual_embedding'] for node in G.nodes() if G.nodes[node]['type'] == 'utterance'])  # 'video' features

adj_normalized = normalized_laplacian(adjacency_matrix)



In [255]:
y_train = [G.nodes[node]['emotion'] for node in G.nodes if G.nodes[node]['type'] == 'emotion']

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

y_train = ohe.fit_transform(np.array(y_train).reshape(-1,1))

y_train = tf.constant(y_train.toarray())

In [256]:
# features_text = tf.constant(np.pad(features_text,((0,5))))
# features_video = tf.constant(np.pad(features_video,((0,5))))
# y_train = tf.constant(np.pad(y_train,((0,9))))

In [257]:
from tensorflow.keras.layers import Input, Dropout, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [258]:
adj_matrix.shape[0]

12

In [259]:
text_in = Input(shape=(features_text.shape[1],))
video_in = Input(shape=(features_video.shape[1],))
adj_in = Input((adj_matrix.shape[0],), sparse=True)

In [260]:

gcn1 = GCNConv(16, activation='relu')([text_in, adj_in])
gcn1 = Dropout(0.5)(gcn1)
gcn2 = GCNConv(8, activation='relu')([gcn1, adj_in])

In [261]:
merged_features = Concatenate()([gcn2, video_in])

In [262]:
fc = Dense(16, activation='relu')(merged_features)
output = Dense(num_classes, activation='softmax')(fc)

In [263]:
model = Model(inputs=[text_in, video_in, adj_in], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [264]:
print(features_text.shape)
print(features_video.shape)
print(y_train.shape)
print(adj_normalized.shape)

(7, 50, 50)
(7, 1000)
(3, 3)
(12, 12)


In [276]:
G.nodes

NodeView(('surprise', 'anger', 'neutral', 'male', 'female', "So in your whole life, you've only been with one\x97\x97oh.", "That's the magic story you use when you wanna have sex!", 'And the nudity is really important to the story.', "Hey Chandler! Y'know that girl you went to college with who-who became a movie director?", 'You really think so?', 'Okay.', "Oh, I know.  Y'know, just-just talking about it is getting me kinda"))

In [274]:
nx.adjacency_matrix(G).toarray()

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [265]:
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 input_31 (InputLayer)          [(None, 12)]         0           []                               
                                                                                                  
 gcn_conv_17 (GCNConv)          (None, 16)           816         ['input_29[0][0]',               
                                                                  'input_31[0][0]']               
                                                                                                  
 dropout_20 (Dropout)           (None, 16)           0           ['gcn_conv_17[0][0]']     

In [266]:
history = model.fit([features_text, features_video, adj_normalized],
          y_train,
          epochs=100,
          batch_size=32,
          )

ValueError: Dimensions 7 and 12 are not compatible